## MISTy pipeline on MIBI data
Run MISTy on cellular measurements made on all fields in the multi-tissue array, as extracted from Mesmer.

In [ ]:
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("mistyR")

In [ ]:
# MISTy
library(mistyR)
library(future)

# data manipulation
library(dplyr)
library(purrr)
library(distances)

# plotting
library(ggplot2)

plan(multisession)

In [ ]:
library(readr)

## Plotting parameters

In [ ]:
library(extrafont)


# Load extra fonts
ttf_import("/tmp/.fonts")
loadfonts()

# Change theme
customTheme <- theme_light() + 
               theme(panel.grid.minor=element_blank(), text=element_text(size=17, family="Arial", colour = "#333333"),
                     line=element_line(colour = "#333333"), 
                     legend.background = element_rect(fill=alpha('#CCCCCC', 0.1)), legend.key = element_blank())

# Change default colors
scale_colour_continuous <- function (..., begin = 0.1, end = 0.9, direction = -1, option = "plasma", 
                                     type = getOption("ggplot2.continuous.colour", default = "viridis")) {
    switch(type, gradient = scale_colour_gradient(...), 
        viridis = scale_colour_viridis_c(option = option, begin = begin, end = end, direction = direction, ...), 
        stop("Unknown scale type", call. = FALSE))
}
scale_color_continuous <- scale_colour_continuous

scale_fill_continuous <- function (..., begin = 0.1, end = 0.9, direction = -1, option = "plasma", 
                                     type = getOption("ggplot2.continuous.colour", default = "viridis")) {
    switch(type, gradient = scale_fill_gradient(...), 
        viridis = scale_fill_viridis_c(option = option, begin = begin, end = end, direction = direction, ...), 
        stop("Unknown scale type", call. = FALSE))

}

cemm_pal = colorRampPalette(c("#5A463C", "#008CAD", "#40B9D4", "#D4ECF2", "#D2323C", "#F8B100", "#DFDC00"))
scale_fill_discrete <- function (..., type = "CeMM", h = c(0, 360) + 15, c = 100, l = 65, h.start = 0, 
    direction = 1, na.value = "grey50", aesthetics = "fill") 
{
    if (type == "CeMM"){
        discrete_scale(aesthetics, "CeMM", cemm_pal, na.value = na.value, ...)
    } else {
        discrete_scale(aesthetics, "hue", hue_pal(h, c, l, h.start, 
            direction), na.value = na.value, ...)
    }
}

scale_color_discrete <- function (..., type = "CeMM", h = c(0, 360) + 15, c = 100, l = 65, h.start = 0, 
    direction = 1, na.value = "grey50", aesthetics = "colour") {
    if (type == "CeMM"){
        discrete_scale(aesthetics, "CeMM", cemm_pal, na.value = na.value, ...)
    } else {
        discrete_scale(aesthetics, "hue", scales::hue_pal(h, c, l, h.start, 
            direction), na.value = na.value, ...)
    }
}
scale_colour_discrete <- scale_color_discrete

noGridTheme <- function(...){
    theme(panel.grid.major=element_blank(), axis.text.x=element_text(size=12), axis.text.y=element_text(size=12),
                      axis.line=element_line(color="#333333", size = 0.2), panel.border = element_blank(), ...)
}

darkTheme <- function(...){
    theme(panel.background = element_rect(fill = '#333333'), plot.background = element_rect(fill = '#333333'), 
          axis.line=element_line(color="#CCCCCC", size = 0.2), 
          text=element_text(size=17, family="Arial", colour = "#CCCCCC"),
          line=element_line(colour = "#CCCCCC"))
}

theme_set(customTheme)

options(repr.plot.width=10, repr.plot.height=10)

## Load data tables

In [ ]:
markers = read_csv("cell_table_size_normalized.csv")

## Define parameters
Adapt this section to your data if needed.

In [ ]:
# Adapt with the columns you want to use to describe each cell
cellfeatures = c('ASCT2', 'ATP5A', 'CD14', 'CD163', 'CD20', 'CD31', 'CD36',
               'CD3e', 'CD4', 'CD45', 'CD45RO', 'CD56', 'CD68', 'CD8', 'CD98',
               'COL1A1', 'CPT1A', 'CS', 'Calprotectin', 'Caveolin', 'ChyTry', 'CytC',
               'EpCAM', 'FoxP3', 'G6PD', 'GLS', 'GLUT1', 'HLADR', 'IL17A', 'Ki67',
               'LDH', 'MCT1', 'PD1', 'PDL1', 'PKM2', 'PanCK', 'SMA', 'Vimentin')

In [ ]:
# Adapt with the columns used as coordinates of each cell
coordfeatures = c("centroid-0", "centroid-1")

In [ ]:
# Folder where the trained MISTy models will be stored
outputfolder = "MISTY_results"

## Define views and run MISTY

In [ ]:
misty.folders = markers %>% 
    group_by(fov) %>% 
    group_map(function(sample, name) {
        print(paste("Processing", name[[1]]))     

        # Format marker table – for now no normalization
        # We drop CD36 has for some slides it's constantly 0
        sample.expr <- sample %>% select(cellfeatures)
        # Extract position
        sample.pos <- sample %>% select(coordfeatures)

        create_initial_view(sample.expr) %>% 
        add_juxtaview(sample.pos, neighbor.thr = 50) %>%
        add_paraview(sample.pos, l = 250, zoi = 50) %>%
        run_misty(results.folder = paste0(outputfolder, .Platform$file.sep, name[[1]]))
    })

## Look at the results

In [ ]:
misty.results <- collect_results(misty.folders)
summary(misty.results)

By default plots are not exported so not customizable. All definitions are taken [from the MISTy GitHub repo](https://github.com/saezlab/mistyR/blob/992c1ac411c95e4d3c57a55d13887b0010b146d3/R/plots.R).

In [ ]:
# Performance of intraview
misty.results %>%
  plot_improvement_stats("intra.R2")

In [ ]:
# Gain compared to intraview

ggplot_improvement_stats <- function(misty.results,
                                   measure = c(
                                     "gain.R2", "multi.R2", "intra.R2",
                                     "gain.RMSE", "multi.RMSE", "intra.RMSE"
                                   ),
                                   trim = -Inf) {
  measure.type <- match.arg(measure)

  assertthat::assert_that(("improvements.stats" %in% names(misty.results)),
    msg = "The provided result list is malformed. Consider using collect_results()."
  )

  inv <- sign((stringr::str_detect(measure.type, "gain") |
    stringr::str_detect(measure.type, "RMSE", negate = TRUE)) - 0.5)

  plot.data <- misty.results$improvements.stats %>%
    dplyr::filter(.data$measure == measure.type, inv * .data$mean >= inv * trim)

  assertthat::assert_that(assertthat::not_empty(plot.data),
    msg = "Invalid selection of measure and/or trim value."
  )

  results.plot <- ggplot2::ggplot(
    plot.data,
    ggplot2::aes(
      x = stats::reorder(.data$target, -.data$mean),
      y = .data$mean
    )
  ) +
    ggplot2::geom_pointrange(ggplot2::aes(
      ymin = .data$mean - .data$sd,
      ymax = .data$mean + .data$sd
    )) +
    ggplot2::geom_point(color = "grey50") +
    ggplot2::ylab(measure) +
    ggplot2::xlab("Target") +
    ggplot2::theme(axis.text.x = ggplot2::element_text(angle = 45, hjust = 1))

  return(results.plot)
}

In [ ]:
plot = misty.results %>%
  ggplot_improvement_stats("gain.R2")
plot + geom_hline(yintercept = 0, lty = 2)

In [ ]:
misty.results %>% plot_interaction_communities("intra")

In [ ]:
misty.results %>% plot_interaction_heatmap(view = "intra", cutoff = 0.8)

Note: fairly robust compared to nuclear markers only, e.g. cluster of intra-interactions between CD3, CD4, CD45 and CD45RO.